In [43]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from layers.Invertible import RevIN
import torch.optim as optim
from torch.optim.lr_scheduler import ExponentialLR
from copy import deepcopy

In [62]:
data = np.load("dataset/Neural_Network_1d_Adam/Sample_1/sample.npy")

In [45]:
data_tr = data[:100,:,:]
data_val = data[100:150,:,:]
data_te = data[150:,:,:]

In [46]:
tr_W1 = data_tr[:,:21,:]
tr_W2 = data_tr[:,21:,:]
val_W1 = data_val[:,:21,:]
val_W2 = data_val[:,21:,:]
te_W1 = data_te[:,:21,:]
te_W2 = data_te[:,21:,:]

In [47]:
tr_W1, tr_W2  = np.transpose(tr_W1, (0,2,1)),np.transpose(tr_W2, (0,2,1))
val_W1, val_W2  = np.transpose(val_W1, (0,2,1)),np.transpose(val_W2, (0,2,1))
te_W1, te_W2  = np.transpose(te_W1, (0,2,1)),np.transpose(te_W2, (0,2,1))

In [59]:
tr_W1.shape, tr_W2.shape 

((100, 3, 21), (100, 3, 180))

In [37]:
# tr_W1, tr_W2  = np.transpose(tr_W1, (0,2,1)),np.transpose(tr_W2, (0,2,1))
# val_W1, val_W2  = np.transpose(val_W1, (0,2,1)),np.transpose(val_W2, (0,2,1))
# te_W1, te_W2  = np.transpose(te_W1, (0,2,1)),np.transpose(te_W2, (0,2,1))

In [48]:
X_tensor = torch.from_numpy(tr_W1).float()
y_tensor = torch.from_numpy(tr_W2).float()
X_tensor_val = torch.from_numpy(val_W1).float()
y_tensor_val = torch.from_numpy(val_W2).float()
X_tensor_te = torch.from_numpy(te_W1).float()
y_tensor_te = torch.from_numpy(te_W2).float()

In [40]:

# class Model(nn.Module):
#     def __init__(self, channel, d_model, seq_len, pred_len):
#         super(Model, self).__init__()
# # Temporal mixing, you can choose any type of extractor such as Attention, Conv 
#         self.temporal = nn.Sequential(
#         nn.Linear(seq_len, d_model),
#         nn.GELU(),
#         nn.Linear(d_model, seq_len)
# )
#         # Channel mixing, you can choose any type of extractor such as MLP, GNN, and et
#         self.channel = nn.Sequential(
#             nn.Linear(channel, d_model),
#             nn.GELU(),
#             nn.Linear(d_model, channel)
#         )
#         self.projection = nn.Linear(seq_len, pred_len)
#         self.rev = RevIN(channel)
# #         self.apply(self._init_weights)
#         # Frozen randomly initialized parameters in temporal feature extractor if neces
#         # for p in self.temporal.parameters():
#         #     p.requires_grad = False
#     # Design your initialization, or random init
# #     def _init_weights(self, m):
# #         if isinstance(m, nn.Linear):
# #             nn.init.xavier_uniform_(m.weight)
# #             if isinstance(m, nn.Linear) and m.bias is not None:
# #                 nn.init.constant_(m.bias, 0)
#     # Design your loss function
#     def forward_loss(self, pred, true):
    
#         return F.mse_loss(pred, true)
#      #   return torch.squeeze(torch.mean(torch.abs(pred - true)))
    
#     def forward(self, x, y): # x: [B, L, D]
# #         x = self.rev(x, 'norm')
# #         x = x + self.temporal(x.transpose(1, 2)).transpose(1, 2)
# #         x = x + self.channel(x)
#         pred = self.projection(x.transpose(1, 2)).transpose(1, 2)
# #         pred = self.rev(pred, 'denorm')
#         return pred, self.forward_loss(pred, y)
# Define the linear regression model


In [61]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt
from copy import deepcopy

# Convert numpy arrays to PyTorch tensors
X_tensor = torch.from_numpy(tr_W1).float()
y_tensor = torch.from_numpy(tr_W2).float()

X_tensor_val = torch.from_numpy(val_W1).float()
y_tensor_val = torch.from_numpy(val_W2).float()

X_tensor_te = torch.from_numpy(te_W1).float()
y_tensor_te = torch.from_numpy(te_W2).float()

# Define the linear regression model
class LinearRegressionModel(nn.Module):
    def __init__(self, input_size,output_size):
        super(LinearRegressionModel, self).__init__()
        self.linear = nn.Linear(input_size, output_size)

    def forward(self, x):
        return self.linear(x)

input_size = tr_W1.shape[2]

output_size = tr_W2.shape[2]

learn_rate = 0.001

model = LinearRegressionModel(input_size,output_size)

# Loss and optimizer
optimizer = optim.Adam(model.parameters(), lr=learn_rate)

# Training the model
num_epochs = 10000

#batchsize
batch_size = 32

best_val_loss = float('inf')
best_model = deepcopy(model.state_dict())
val_loss_list = [] 
impatience = 0 

for epoch in range(num_epochs):
    # Forward pass
    indices = torch.randperm(len(X_tensor))

    for i in range(0, len(X_tensor), batch_size):
        batch_indices = indices[i:i+batch_size]
    
        outputs = model( X_tensor[batch_indices])
        
        # Compute the loss : 
        loss = torch.squeeze(torch.mean(torch.abs(outputs - y_tensor[batch_indices])))
        
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
    tr_loss = torch.squeeze(torch.mean(torch.abs(model(X_tensor) - y_tensor)))
    val_loss= torch.squeeze(torch.mean(torch.abs(model(X_tensor_val) - y_tensor_val)))
    print(f'Epoch [{epoch+1}/{num_epochs}], Train Loss: {tr_loss.item():.10f}, val Loss: {val_loss.item():.10f} ')

    if  val_loss < best_val_loss:
            best_val_loss = val_loss
            best_model = deepcopy(model.state_dict())
            impatient = 0 # reset
    else:
        impatient += 1


    if impatient >= 5:
        print(f'Breaking due to early stopping at epoch {epoch}')
        break

Epoch [1/10000], Train Loss: 0.5352321267, val Loss: 0.5815230608 
Epoch [2/10000], Train Loss: 0.5146657228, val Loss: 0.5622635484 
Epoch [3/10000], Train Loss: 0.4960346222, val Loss: 0.5449951291 
Epoch [4/10000], Train Loss: 0.4790568650, val Loss: 0.5295261741 
Epoch [5/10000], Train Loss: 0.4640507102, val Loss: 0.5159592628 
Epoch [6/10000], Train Loss: 0.4508683383, val Loss: 0.5042133331 
Epoch [7/10000], Train Loss: 0.4393232167, val Loss: 0.4940155149 
Epoch [8/10000], Train Loss: 0.4294627607, val Loss: 0.4853464961 
Epoch [9/10000], Train Loss: 0.4210088849, val Loss: 0.4779731035 
Epoch [10/10000], Train Loss: 0.4137648046, val Loss: 0.4716607332 
Epoch [11/10000], Train Loss: 0.4075873196, val Loss: 0.4663417339 
Epoch [12/10000], Train Loss: 0.4024111927, val Loss: 0.4619101584 
Epoch [13/10000], Train Loss: 0.3980136812, val Loss: 0.4581903815 
Epoch [14/10000], Train Loss: 0.3941704333, val Loss: 0.4548469782 
Epoch [15/10000], Train Loss: 0.3908239603, val Loss: 0.4

Epoch [128/10000], Train Loss: 0.2745141685, val Loss: 0.3183258772 
Epoch [129/10000], Train Loss: 0.2735657096, val Loss: 0.3172780275 
Epoch [130/10000], Train Loss: 0.2725767195, val Loss: 0.3162184656 
Epoch [131/10000], Train Loss: 0.2715689540, val Loss: 0.3150579929 
Epoch [132/10000], Train Loss: 0.2705734670, val Loss: 0.3137933910 
Epoch [133/10000], Train Loss: 0.2695687711, val Loss: 0.3126300871 
Epoch [134/10000], Train Loss: 0.2686028481, val Loss: 0.3114535213 
Epoch [135/10000], Train Loss: 0.2676407993, val Loss: 0.3102175295 
Epoch [136/10000], Train Loss: 0.2667988539, val Loss: 0.3090152144 
Epoch [137/10000], Train Loss: 0.2659705281, val Loss: 0.3078314960 
Epoch [138/10000], Train Loss: 0.2651075423, val Loss: 0.3067105412 
Epoch [139/10000], Train Loss: 0.2641441524, val Loss: 0.3057295680 
Epoch [140/10000], Train Loss: 0.2632294297, val Loss: 0.3046375215 
Epoch [141/10000], Train Loss: 0.2623172104, val Loss: 0.3036214113 
Epoch [142/10000], Train Loss: 0.2

Epoch [253/10000], Train Loss: 0.1619104594, val Loss: 0.1862457693 
Epoch [254/10000], Train Loss: 0.1609930843, val Loss: 0.1852196157 
Epoch [255/10000], Train Loss: 0.1603280902, val Loss: 0.1844095737 
Epoch [256/10000], Train Loss: 0.1596356630, val Loss: 0.1835837066 
Epoch [257/10000], Train Loss: 0.1586159617, val Loss: 0.1824122965 
Epoch [258/10000], Train Loss: 0.1576479375, val Loss: 0.1813358068 
Epoch [259/10000], Train Loss: 0.1567854434, val Loss: 0.1803912073 
Epoch [260/10000], Train Loss: 0.1560965925, val Loss: 0.1795639992 
Epoch [261/10000], Train Loss: 0.1553443670, val Loss: 0.1786813587 
Epoch [262/10000], Train Loss: 0.1544973254, val Loss: 0.1776940674 
Epoch [263/10000], Train Loss: 0.1536042392, val Loss: 0.1766649783 
Epoch [264/10000], Train Loss: 0.1528923362, val Loss: 0.1758108139 
Epoch [265/10000], Train Loss: 0.1520638019, val Loss: 0.1748674661 
Epoch [266/10000], Train Loss: 0.1511642933, val Loss: 0.1737976819 
Epoch [267/10000], Train Loss: 0.1

Epoch [381/10000], Train Loss: 0.0716701895, val Loss: 0.0817784742 
Epoch [382/10000], Train Loss: 0.0711001679, val Loss: 0.0811607689 
Epoch [383/10000], Train Loss: 0.0707646832, val Loss: 0.0809276253 
Epoch [384/10000], Train Loss: 0.0702811927, val Loss: 0.0803702325 
Epoch [385/10000], Train Loss: 0.0697059929, val Loss: 0.0795637816 
Epoch [386/10000], Train Loss: 0.0690592974, val Loss: 0.0787630752 
Epoch [387/10000], Train Loss: 0.0685824901, val Loss: 0.0781766847 
Epoch [388/10000], Train Loss: 0.0681088865, val Loss: 0.0777585730 
Epoch [389/10000], Train Loss: 0.0676744804, val Loss: 0.0773546249 
Epoch [390/10000], Train Loss: 0.0670672730, val Loss: 0.0765574798 
Epoch [391/10000], Train Loss: 0.0668038577, val Loss: 0.0760327578 
Epoch [392/10000], Train Loss: 0.0668940395, val Loss: 0.0760655999 
Epoch [393/10000], Train Loss: 0.0659215301, val Loss: 0.0750243515 
Epoch [394/10000], Train Loss: 0.0652595982, val Loss: 0.0742702782 
Epoch [395/10000], Train Loss: 0.0

Epoch [516/10000], Train Loss: 0.0314429291, val Loss: 0.0350153260 
Epoch [517/10000], Train Loss: 0.0314662494, val Loss: 0.0350136980 
Epoch [518/10000], Train Loss: 0.0311653875, val Loss: 0.0346942134 
Epoch [519/10000], Train Loss: 0.0310079437, val Loss: 0.0345421284 
Epoch [520/10000], Train Loss: 0.0313390680, val Loss: 0.0349232145 
Epoch [521/10000], Train Loss: 0.0311457030, val Loss: 0.0347145312 
Epoch [522/10000], Train Loss: 0.0307719857, val Loss: 0.0342735425 
Epoch [523/10000], Train Loss: 0.0306562446, val Loss: 0.0339768454 
Epoch [524/10000], Train Loss: 0.0308238845, val Loss: 0.0340313688 
Epoch [525/10000], Train Loss: 0.0306291766, val Loss: 0.0338420868 
Epoch [526/10000], Train Loss: 0.0304324701, val Loss: 0.0337683186 
Epoch [527/10000], Train Loss: 0.0303349476, val Loss: 0.0337058268 
Epoch [528/10000], Train Loss: 0.0303087197, val Loss: 0.0335476995 
Epoch [529/10000], Train Loss: 0.0302217994, val Loss: 0.0334305651 
Epoch [530/10000], Train Loss: 0.0

In [58]:
model(X_tensor_te).shape

torch.Size([50, 3, 180])

In [54]:
# evaluate
model.load_state_dict(best_model)
    
model.eval()
for k in [2,3,4,5,6,7,8,9,10]:
    mse = torch.mean(torch.squeeze(torch.square(model(X_tensor_te)[:,:,20*(k-1)-1] - y_tensor_te[:,:,20*(k-1)-1])))

    print("mse of test samples {} at predicting time step {} ".format(mse, k*20 ) )

mse of test samples 0.00025497135356999934 at predicting time step 40 
mse of test samples 0.0001347107463516295 at predicting time step 60 
mse of test samples 0.0004435234295669943 at predicting time step 80 
mse of test samples 0.0010326358024030924 at predicting time step 100 
mse of test samples 0.0018329339800402522 at predicting time step 120 
mse of test samples 0.0033399565145373344 at predicting time step 140 
mse of test samples 0.005522863008081913 at predicting time step 160 
mse of test samples 0.00653043994680047 at predicting time step 180 
mse of test samples 0.008375406265258789 at predicting time step 200 


In [41]:

np.random.seed(0)
torch.manual_seed(0)

model = Model(channel= X_tensor.shape[2], d_model=512, seq_len=X_tensor.shape[1], pred_len=y_tensor.shape[1])

learn_rate = 0.0001
# Loss and optimizer
optimizer = optim.Adam(model.parameters(), lr=learn_rate)
#scheduler = ExponentialLR(optimizer, gamma=1)
# Training the model
num_epochs = 10000 #batchsize
batch_size = 32


best_val_loss = float('inf')
best_model = deepcopy(model.state_dict())
val_loss_list = [] 
impatience = 0 


for epoch in range(num_epochs):
    # Forward pass
    indices = torch.randperm(len(X_tensor))

    for i in range(0, len(X_tensor), batch_size):
        batch_indices = indices[i:i+batch_size]
    
        _, loss = model( X_tensor[batch_indices], y_tensor[batch_indices] )

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
    
    _, tr_loss = model(X_tensor, y_tensor)
    _, val_loss= model(X_tensor_val, y_tensor_val)
    print(f'Epoch [{epoch+1}/{num_epochs}], Train Loss: {tr_loss.item():.10f}, val Loss: {val_loss.item():.10f} ')

    if  val_loss < best_val_loss:
            best_val_loss = val_loss
            best_model = deepcopy(model.state_dict())
            impatient = 0 # reset
    else:
        impatient += 1


    if impatient >= 20:
        print(f'Breaking due to early stopping at epoch {epoch}')
        break

Epoch [1/10000], Train Loss: 0.4793511331, val Loss: 0.5689533353 
Epoch [2/10000], Train Loss: 0.4756829441, val Loss: 0.5653091669 
Epoch [3/10000], Train Loss: 0.4721337855, val Loss: 0.5617802739 
Epoch [4/10000], Train Loss: 0.4686393142, val Loss: 0.5583044291 
Epoch [5/10000], Train Loss: 0.4652186036, val Loss: 0.5549000502 
Epoch [6/10000], Train Loss: 0.4619427025, val Loss: 0.5516376495 
Epoch [7/10000], Train Loss: 0.4586352706, val Loss: 0.5483453274 
Epoch [8/10000], Train Loss: 0.4553564489, val Loss: 0.5450825095 
Epoch [9/10000], Train Loss: 0.4521043897, val Loss: 0.5418487191 
Epoch [10/10000], Train Loss: 0.4487985075, val Loss: 0.5385636687 
Epoch [11/10000], Train Loss: 0.4454633892, val Loss: 0.5352514982 
Epoch [12/10000], Train Loss: 0.4422079027, val Loss: 0.5320147276 
Epoch [13/10000], Train Loss: 0.4390713871, val Loss: 0.5288932920 
Epoch [14/10000], Train Loss: 0.4359911084, val Loss: 0.5258283615 
Epoch [15/10000], Train Loss: 0.4330112040, val Loss: 0.5

Epoch [152/10000], Train Loss: 0.2681283653, val Loss: 0.3562483490 
Epoch [153/10000], Train Loss: 0.2677627802, val Loss: 0.3558539152 
Epoch [154/10000], Train Loss: 0.2674100101, val Loss: 0.3554678559 
Epoch [155/10000], Train Loss: 0.2670731843, val Loss: 0.3550948799 
Epoch [156/10000], Train Loss: 0.2667233050, val Loss: 0.3547113240 
Epoch [157/10000], Train Loss: 0.2663839459, val Loss: 0.3543366492 
Epoch [158/10000], Train Loss: 0.2660487890, val Loss: 0.3539688885 
Epoch [159/10000], Train Loss: 0.2657136619, val Loss: 0.3536044061 
Epoch [160/10000], Train Loss: 0.2653905749, val Loss: 0.3532496393 
Epoch [161/10000], Train Loss: 0.2650831044, val Loss: 0.3529077768 
Epoch [162/10000], Train Loss: 0.2647618651, val Loss: 0.3525519967 
Epoch [163/10000], Train Loss: 0.2644299269, val Loss: 0.3521907926 
Epoch [164/10000], Train Loss: 0.2640990019, val Loss: 0.3518340886 
Epoch [165/10000], Train Loss: 0.2637794614, val Loss: 0.3514879048 
Epoch [166/10000], Train Loss: 0.2

Epoch [271/10000], Train Loss: 0.2425102293, val Loss: 0.3263145685 
Epoch [272/10000], Train Loss: 0.2423698604, val Loss: 0.3261444867 
Epoch [273/10000], Train Loss: 0.2422318608, val Loss: 0.3259769678 
Epoch [274/10000], Train Loss: 0.2420863211, val Loss: 0.3257955313 
Epoch [275/10000], Train Loss: 0.2419345677, val Loss: 0.3255929649 
Epoch [276/10000], Train Loss: 0.2417890280, val Loss: 0.3254033625 
Epoch [277/10000], Train Loss: 0.2416430414, val Loss: 0.3252147138 
Epoch [278/10000], Train Loss: 0.2414966673, val Loss: 0.3250253797 
Epoch [279/10000], Train Loss: 0.2413437515, val Loss: 0.3248265088 
Epoch [280/10000], Train Loss: 0.2411973178, val Loss: 0.3246384859 
Epoch [281/10000], Train Loss: 0.2410490513, val Loss: 0.3244456053 
Epoch [282/10000], Train Loss: 0.2409043014, val Loss: 0.3242542446 
Epoch [283/10000], Train Loss: 0.2407649308, val Loss: 0.3240727782 
Epoch [284/10000], Train Loss: 0.2406305671, val Loss: 0.3239011168 
Epoch [285/10000], Train Loss: 0.2

Epoch [393/10000], Train Loss: 0.2261304557, val Loss: 0.3044421971 
Epoch [394/10000], Train Loss: 0.2260039747, val Loss: 0.3042782545 
Epoch [395/10000], Train Loss: 0.2258707285, val Loss: 0.3041042984 
Epoch [396/10000], Train Loss: 0.2257396579, val Loss: 0.3039329052 
Epoch [397/10000], Train Loss: 0.2256087214, val Loss: 0.3037569523 
Epoch [398/10000], Train Loss: 0.2254759073, val Loss: 0.3035749495 
Epoch [399/10000], Train Loss: 0.2253332734, val Loss: 0.3033798933 
Epoch [400/10000], Train Loss: 0.2251970470, val Loss: 0.3032002151 
Epoch [401/10000], Train Loss: 0.2250632942, val Loss: 0.3030235767 
Epoch [402/10000], Train Loss: 0.2249294668, val Loss: 0.3028454483 
Epoch [403/10000], Train Loss: 0.2247987241, val Loss: 0.3026632965 
Epoch [404/10000], Train Loss: 0.2246632725, val Loss: 0.3024836481 
Epoch [405/10000], Train Loss: 0.2245326489, val Loss: 0.3022997379 
Epoch [406/10000], Train Loss: 0.2243994027, val Loss: 0.3021115959 
Epoch [407/10000], Train Loss: 0.2

Epoch [515/10000], Train Loss: 0.2108514905, val Loss: 0.2838554978 
Epoch [516/10000], Train Loss: 0.2107340991, val Loss: 0.2837021947 
Epoch [517/10000], Train Loss: 0.2106177658, val Loss: 0.2835532725 
Epoch [518/10000], Train Loss: 0.2104978114, val Loss: 0.2833975852 
Epoch [519/10000], Train Loss: 0.2103780061, val Loss: 0.2832332253 
Epoch [520/10000], Train Loss: 0.2102604359, val Loss: 0.2830737829 
Epoch [521/10000], Train Loss: 0.2101376057, val Loss: 0.2829031348 
Epoch [522/10000], Train Loss: 0.2100123316, val Loss: 0.2827227414 
Epoch [523/10000], Train Loss: 0.2098913342, val Loss: 0.2825475633 
Epoch [524/10000], Train Loss: 0.2097697109, val Loss: 0.2823710740 
Epoch [525/10000], Train Loss: 0.2096435577, val Loss: 0.2821916044 
Epoch [526/10000], Train Loss: 0.2095226943, val Loss: 0.2820262909 
Epoch [527/10000], Train Loss: 0.2093957961, val Loss: 0.2818464637 
Epoch [528/10000], Train Loss: 0.2092710733, val Loss: 0.2816736400 
Epoch [529/10000], Train Loss: 0.2

...

Epoch [5131/10000], Train Loss: 0.0021091427, val Loss: 0.0025204013 
Epoch [5132/10000], Train Loss: 0.0021107146, val Loss: 0.0025194432 
Epoch [5133/10000], Train Loss: 0.0021124147, val Loss: 0.0025191791 
Epoch [5134/10000], Train Loss: 0.0021098191, val Loss: 0.0025182420 
Epoch [5135/10000], Train Loss: 0.0021084903, val Loss: 0.0025175372 
Epoch [5136/10000], Train Loss: 0.0021116622, val Loss: 0.0025178595 
Epoch [5137/10000], Train Loss: 0.0021114904, val Loss: 0.0025171442 
Epoch [5138/10000], Train Loss: 0.0021068417, val Loss: 0.0025151819 
Epoch [5139/10000], Train Loss: 0.0021032628, val Loss: 0.0025146543 
Epoch [5140/10000], Train Loss: 0.0021016151, val Loss: 0.0025140524 
Epoch [5141/10000], Train Loss: 0.0021001948, val Loss: 0.0025152422 
Epoch [5142/10000], Train Loss: 0.0020992870, val Loss: 0.0025134382 
Epoch [5143/10000], Train Loss: 0.0020987354, val Loss: 0.0025119283 
Epoch [5144/10000], Train Loss: 0.0020981554, val Loss: 0.0025107563 
Epoch [5145/10000], 

Epoch [5250/10000], Train Loss: 0.0020416128, val Loss: 0.0024382453 
Epoch [5251/10000], Train Loss: 0.0020421450, val Loss: 0.0024412407 
Epoch [5252/10000], Train Loss: 0.0020411874, val Loss: 0.0024395143 
Epoch [5253/10000], Train Loss: 0.0020400984, val Loss: 0.0024367154 
Epoch [5254/10000], Train Loss: 0.0020395827, val Loss: 0.0024360956 
Epoch [5255/10000], Train Loss: 0.0020391431, val Loss: 0.0024356525 
Epoch [5256/10000], Train Loss: 0.0020388870, val Loss: 0.0024358896 
Epoch [5257/10000], Train Loss: 0.0020388961, val Loss: 0.0024367690 
Epoch [5258/10000], Train Loss: 0.0020381419, val Loss: 0.0024353219 
Epoch [5259/10000], Train Loss: 0.0020371862, val Loss: 0.0024314516 
Epoch [5260/10000], Train Loss: 0.0020368821, val Loss: 0.0024295321 
Epoch [5261/10000], Train Loss: 0.0020368199, val Loss: 0.0024280827 
Epoch [5262/10000], Train Loss: 0.0020358900, val Loss: 0.0024293666 
Epoch [5263/10000], Train Loss: 0.0020370854, val Loss: 0.0024352225 
Epoch [5264/10000], 

Epoch [5373/10000], Train Loss: 0.0019966280, val Loss: 0.0023779937 
Epoch [5374/10000], Train Loss: 0.0019972986, val Loss: 0.0023808279 
Epoch [5375/10000], Train Loss: 0.0019996925, val Loss: 0.0023863250 
Epoch [5376/10000], Train Loss: 0.0020005833, val Loss: 0.0023883097 
Epoch [5377/10000], Train Loss: 0.0020009228, val Loss: 0.0023890266 
Epoch [5378/10000], Train Loss: 0.0020046495, val Loss: 0.0023955610 
Epoch [5379/10000], Train Loss: 0.0020037706, val Loss: 0.0023943996 
Epoch [5380/10000], Train Loss: 0.0020003780, val Loss: 0.0023888634 
Epoch [5381/10000], Train Loss: 0.0019963232, val Loss: 0.0023807776 
Epoch [5382/10000], Train Loss: 0.0019943959, val Loss: 0.0023760344 
Epoch [5383/10000], Train Loss: 0.0019936040, val Loss: 0.0023724812 
Epoch [5384/10000], Train Loss: 0.0019930713, val Loss: 0.0023720008 
Epoch [5385/10000], Train Loss: 0.0019927055, val Loss: 0.0023727850 
Epoch [5386/10000], Train Loss: 0.0019923667, val Loss: 0.0023715335 
Epoch [5387/10000], 

Epoch [5506/10000], Train Loss: 0.0019661500, val Loss: 0.0023357747 
Epoch [5507/10000], Train Loss: 0.0019664867, val Loss: 0.0023389237 
Epoch [5508/10000], Train Loss: 0.0019690306, val Loss: 0.0023457501 
Epoch [5509/10000], Train Loss: 0.0019755380, val Loss: 0.0023578997 
Epoch [5510/10000], Train Loss: 0.0019781983, val Loss: 0.0023627204 
Epoch [5511/10000], Train Loss: 0.0019731163, val Loss: 0.0023545073 
Epoch [5512/10000], Train Loss: 0.0019698376, val Loss: 0.0023483974 
Epoch [5513/10000], Train Loss: 0.0019666592, val Loss: 0.0023394399 
Epoch [5514/10000], Train Loss: 0.0019669430, val Loss: 0.0023341745 
Epoch [5515/10000], Train Loss: 0.0019686951, val Loss: 0.0023319360 
Epoch [5516/10000], Train Loss: 0.0019693212, val Loss: 0.0023310259 
Epoch [5517/10000], Train Loss: 0.0019696353, val Loss: 0.0023305956 
Epoch [5518/10000], Train Loss: 0.0019705086, val Loss: 0.0023304906 
Epoch [5519/10000], Train Loss: 0.0019701130, val Loss: 0.0023303437 
Epoch [5520/10000], 

Epoch [5646/10000], Train Loss: 0.0019499102, val Loss: 0.0023223599 
Epoch [5647/10000], Train Loss: 0.0019497178, val Loss: 0.0023224363 
Epoch [5648/10000], Train Loss: 0.0019489193, val Loss: 0.0023209876 
Epoch [5649/10000], Train Loss: 0.0019487189, val Loss: 0.0023209979 
Epoch [5650/10000], Train Loss: 0.0019494093, val Loss: 0.0023230934 
Epoch [5651/10000], Train Loss: 0.0019490072, val Loss: 0.0023225558 
Epoch [5652/10000], Train Loss: 0.0019484030, val Loss: 0.0023211841 
Epoch [5653/10000], Train Loss: 0.0019474535, val Loss: 0.0023180568 
Epoch [5654/10000], Train Loss: 0.0019469962, val Loss: 0.0023153333 
Breaking due to early stopping at epoch 5653


In [55]:
# evaluate
model.load_state_dict(best_model)
    
model.eval()

pred,_ = model(X_tensor_te,y_tensor_te)

for k in [3,4,5,6,7,8,9,10]:
    mse = torch.mean(torch.squeeze(torch.square(pred[:,20*(k-1)-1,:] - y_tensor_te[:,20*(k-1)-1,:])))

    print("mse of test samples {} at predicting time step {} ".format(mse, k*20 ) )


TypeError: LinearRegressionModel.forward() takes 2 positional arguments but 3 were given